<a href="https://colab.research.google.com/github/virajvaidya/Australia-Energy-Demand-Forecasting/blob/main/NSWPIPELINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas==1.3.5

In [ ]:
!pip install plotly==5.4.0
!pip install chart_studio

In [ ]:
!pip install statsmodels==0.13.1

In [ ]:
!pip install pmdarima

In [5]:
import pandas       as pd
import numpy        as np
import pmdarima     as pm
from pmdarima       import pipeline
from pmdarima       import model_selection
from pmdarima       import preprocessing as ppc
from pmdarima       import arima
from matplotlib     import pyplot as plt
import seaborn      as sns

In [6]:
import plotly.graph_objs    as go
from plotly.offline                       import iplot
from plotly.subplots                      import make_subplots

In [7]:
sns.set_style('whitegrid')
pd.plotting.register_matplotlib_converters()
# Default figure size
sns.mpl.rc('figure',figsize=(16, 6))

In [8]:
from google.colab import files
import io
uploaded = files.upload()

Saving NSW.csv to NSW.csv


In [9]:
df = pd.read_csv('NSW.csv')

In [10]:
df= df[['DATE','TOTALDEMANDNSW','RRPNSW']]

In [11]:
df['DATE'] = df['DATE'].apply(lambda x: pd.Timestamp(x).strftime('%d/%m/%Y %H%M'))

In [12]:
df['DATE'] = df['DATE'].astype('datetime64[ns]')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70942 entries, 0 to 70941
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DATE            70942 non-null  datetime64[ns]
 1   TOTALDEMANDNSW  70942 non-null  float64       
 2   RRPNSW          70942 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.6+ MB


In [15]:
data = df['TOTALDEMANDNSW']
train, test = model_selection.train_test_split(data, train_size=40000)

In [16]:
pipe = pipeline.Pipeline([
    ("fourier", ppc.FourierFeaturizer(m=12, k=4)),
    ("arima", arima.AutoARIMA(stepwise=True, trace=1, error_action="ignore",
                              information_criterion='bic',
                              seasonal=False,  # because we use Fourier
                              suppress_warnings=True))
])

In [17]:
pipe.fit(train)
print("Model fit:")
print(pipe)

Performing stepwise search to minimize bic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : BIC=486626.239, Time=99.28 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : BIC=540991.806, Time=3.44 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : BIC=491458.080, Time=8.46 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : BIC=514499.991, Time=32.07 sec
 ARIMA(0,1,0)(0,0,0)[0]             : BIC=540981.538, Time=28.09 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : BIC=489251.932, Time=52.27 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : BIC=491411.569, Time=61.01 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : BIC=486262.890, Time=103.51 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : BIC=486563.104, Time=87.86 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : BIC=486584.098, Time=150.41 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : BIC=485665.566, Time=162.25 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : BIC=485655.612, Time=100.02 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : BIC=489025.101, Time=62.25 sec
 ARIMA(2,1,4)(0,0,0)[0] intercept   : BIC=485665.504, Time

In [18]:
preds, conf_int = pipe.predict(n_periods=100, return_conf_int=True)
print("\nForecasts:")
print(preds)


Forecasts:
[6940.73256638 6640.4846873  6391.1577404  6230.05717799 6131.93936183
 6091.70742525 6125.39659436 6171.34192515 6231.69247537 6262.16975178
 6305.0067797  6338.75219304 6385.80935608 6456.00761752 6544.49396028
 6679.40900513 6809.18271819 6964.73633549 7110.94724017 7242.95988239
 7315.85019892 7329.64500445 7311.16967751 7247.43094911 7187.046497
 7111.46944203 7070.27183117 7046.94472867 7043.61935295 7059.37893667
 7084.19418913 7114.46593506 7090.50232024 7046.61179794 6966.09090735
 6883.46769304 6801.42739092 6734.70919853 6710.69877833 6712.89669619
 6756.17904284 6806.57262937 6892.26679639 6962.46397538 6997.37119402
 6996.3491125  6960.40211112 6920.03353961 6863.27199749 6832.38015829
 6816.67322976 6841.57837828 6882.81791527 6938.21274052 7018.17746212
 7075.87420022 7104.79648152 7079.38574665 7036.70769968 6969.04692
 6902.27286097 6849.80180393 6819.22883812 6833.01649248 6856.08892424
 6910.25986202 6973.3983384  7035.14118793 7054.11034679 7032.47385952